In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
#Loading dataset

In [ ]:
data_ebill = pd.read_csv('11kV bill_units.csv')
data_ebill

In [ ]:
data_ebill.loc[27,'Units_kWh']=358912
data_ebill.loc[25,'Units_kWh']=378448

In [ ]:
data_ebill = data_ebill[data_ebill.Month.notna()]

In [ ]:
data_ebill.shape

In [ ]:
data_ebill.columns

In [ ]:
data_ebill.dtypes

In [ ]:
data_ebill.info()

In [ ]:
data_ebill.Year = data_ebill.Year.astype(int).astype(str)
data_ebill.head()

In [ ]:
data_ebill['Standardized_Date'] = data_ebill['Month']+'-'+data_ebill['Year']
data_ebill.head()

In [ ]:
data_ebill['Standardized_Date'] = pd.to_datetime(data_ebill['Standardized_Date'])+ pd.offsets.MonthEnd(0)
data_ebill.head()

In [ ]:
data = pd.read_csv('final_data_in_ML.csv',index_col='Standardized_Date',parse_dates=True)
data

In [ ]:
data_month_start = data.resample('M').agg({'RAW WATER FLOW IN ML':'sum',
                                   'CLEAR WATER SUMP LEVEL IN Meter':'mean',
                                   'CLEAR WATER PUMPING FLOW ML':'sum',
                                   'TREATED WATER PRODUCTION IN ML':'sum', 'remarks category':'unique'}).reset_index()
data_month_start.head()

In [ ]:
data_month_ebill = data_month_start.merge(data_ebill,on='Standardized_Date')
data_month_ebill.shape

In [ ]:
data_month_ebill = data_month_ebill[['Standardized_Date','RAW WATER FLOW IN ML',
                                   'CLEAR WATER SUMP LEVEL IN Meter',
                                   'CLEAR WATER PUMPING FLOW ML',
                                   'TREATED WATER PRODUCTION IN ML','remarks category',
                                    'Avg_Consumption (kWh)','Energy Charge','Units_kWh']]
data_month_ebill

In [ ]:
data_month_ebill.dtypes

In [ ]:
data_month_ebill['Energy Charge'] = data_month_ebill['Energy Charge'].astype(float)

In [ ]:
# data_month_ebill['charge_per_unit'] = data_month_ebill['Energy Charge']/ data_month_ebill['Units_kWh']

In [ ]:
## Specific energy consumption: amount of units taken to produce 1ML of treated water
# data_month_ebill['specific_energy_consumption'] = (data_month_ebill['Units_kWh']/data_month_ebill['TREATED WATER PRODUCTION IN ML']).round(2)
## unit_cost: the cost for producing 1ML of treated water
# data_month_ebill['unit_cost'] = (data_month_ebill['Energy Charge']/data_month_ebill['TREATED WATER PRODUCTION IN ML']).round(2)
# data_month_ebill

In [ ]:
data_month_ebill['Year'] = data_month_ebill['Standardized_Date'].dt.year
data_month_ebill['Month'] = data_month_ebill['Standardized_Date'].dt.month
data_month_ebill['Month-Year'] = data_month_ebill['Standardized_Date'].dt.strftime('%m-%Y')

In [ ]:
data_month_ebill.head()

In [ ]:
data_month_ebill.dtypes

In [ ]:
# data_month_ebill1 = data_month_ebill.fillna(data_month_ebill.rolling(3, min_periods=1, center=True).mean())

In [ ]:
# data_month_ebill == data_month_ebill1

In [ ]:
plt.figure(figsize=(15,12))

plt.subplot(2,2,1)
plt.plot(data_month_ebill[data_month_ebill['Year']==2022]['Month'], data_month_ebill[data_month_ebill['Year']==2022]['Units_kWh'], label='Consumption', marker='o',color='g')
plt.xticks(rotation=45)
plt.title("2022")
plt.legend()

plt.subplot(2,2,2)
plt.plot(data_month_ebill[data_month_ebill['Year']==2022]['Month'], data_month_ebill[data_month_ebill['Year']==2022]['CLEAR WATER PUMPING FLOW ML'], label='Clear water pumping flow', marker='o',color='r')
plt.xticks(rotation=45)
plt.title("2022")
plt.legend()

plt.subplot(2,2,3)
plt.plot(data_month_ebill[data_month_ebill['Year']==2023]['Month'], data_month_ebill[data_month_ebill['Year']==2023]['Units_kWh'], label='Consumption', marker='o',color='b')
plt.xticks(rotation=45)
plt.title("2023")
plt.legend()

plt.subplot(2,2,4)
plt.plot(data_month_ebill[data_month_ebill['Year']==2023]['Month'], data_month_ebill[data_month_ebill['Year']==2023]['CLEAR WATER PUMPING FLOW ML'], label='Clear water pumping flow', marker='o',color='y')
plt.xticks(rotation=45)
plt.title("2023")
plt.legend()

plt.show()

In [ ]:
import plotly.express as px

In [ ]:
x = data_month_ebill[data_month_ebill['Year']==2022]['Month']
y1 = (data_month_ebill[data_month_ebill['Year']==2022]['Units_kWh'])/100
y2 = data_month_ebill[data_month_ebill['Year']==2022]['CLEAR WATER PUMPING FLOW ML']

fig = px.bar(data_month_ebill, x=x, y=[y1, y2], barmode='group') #title='Raw Water Pumped during various zones in ML')
fig.show()

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()

In [ ]:
# data_month_ebill[['Clear Water Pumping scale', 'Consumption scale']] = scaler.fit_transform(data_month_ebill[['CLEAR WATER PUMPING FLOW ML', 'Units_kWh']])

In [ ]:
data_month_ebill

In [ ]:
data_month_ebill.to_csv('data_month_ebill_cleaned.csv',index=False)

In [ ]:
data_ebill_2022 = data_month_ebill[data_month_ebill['Year']==2022]
# y1 = data_month_ebill[data_month_ebill['Year']==2022]['Consumption scale']
# y2 = data_month_ebill[data_month_ebill['Year']==2022]['Clear Water Pumping scale']
fig = px.bar(data_ebill_2022, x='Month', y=['Consumption scale', 'Clear Water Pumping scale'], barmode='group') #title='Raw Water Pumped during various zones in ML')

fig.show()

In [ ]:
data_ebill_2022 = data_month_ebill[data_month_ebill['Year']==2023]
# y1 = data_month_ebill[data_month_ebill['Year']==2022]['Consumption scale']
# y2 = data_month_ebill[data_month_ebill['Year']==2022]['Clear Water Pumping scale']
fig = px.bar(data_ebill_2022, x='Month', y=['Consumption scale', 'Clear Water Pumping scale'], barmode='group') #title='Raw Water Pumped during various zones in ML')

fig.show()

In [ ]:
data_ebill_2022_2023 = data_month_ebill[(data_month_ebill['Year']==2022)|(data_month_ebill['Year']==2023)]
fig = px.bar(data_ebill_2022_2023, x='Month-Year', y=['Consumption scale', 'Clear Water Pumping scale'], barmode='group') #title='Raw Water Pumped during various zones in ML')

fig.show()

In [ ]:
data_ebill_2022_2023 = data_month_ebill[(data_month_ebill['Year']==2022)|(data_month_ebill['Year']==2023)]
fig = px.bar(data_ebill_2022_2023, x='Month-Year', y=['Consumption scale', 'Clear Water Pumping scale'], barmode='group') #title='Raw Water Pumped during various zones in ML')

fig.show()

In [ ]:
sorted_data = data_month_ebill.sort_values(by='CLEAR WATER PUMPING FLOW ML')

fig = px.line(sorted_data, x="CLEAR WATER PUMPING FLOW ML", y="Units_kWh", hover_data=['Standardized_Date','RAW WATER FLOW IN ML'])
fig.show()

In [ ]:
data_month_ebill.dtypes

In [ ]:
data[(data.index>='2022-03-01')&(data.index<'2022-04-01')&(data['remarks category']!='No remarks')].shape

In [ ]:
data[(data.index>='2022-04-01')&(data.index<'2022-05-01')&(data['remarks category']!='No remarks')]#.shape

In [ ]:
data[(data.index>='2023-10-01')&(data.index<'2023-11-01')&(data['remarks category']!='No remarks')].shape

In [ ]:
data[(data.index>='2023-12-01')&(data.index<'2024-01-01')&(data['remarks category']!='No remarks')].shape

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(data_month_ebill[data_month_ebill['Standardized_Date']<'2023-01-01']['Standardized_Date'], (data_month_ebill[data_month_ebill['Standardized_Date']<'2023-01-01']['Units_kWh'])/100, label='consumption', marker='o',color='g')
plt.plot(data_month_ebill[data_month_ebill['Standardized_Date']<'2023-01-01']['Standardized_Date'], data_month_ebill[data_month_ebill['Standardized_Date']<'2023-01-01']['CLEAR WATER PUMPING FLOW ML'], label='clear water', marker='o',color='r')
plt.legend()
plt.xticks(rotation=45)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(data_month_ebill[(data_month_ebill['Standardized_Date']>='2023-01-01')&(data_month_ebill['Standardized_Date']<'2024-01-01')]['Standardized_Date'], (data_month_ebill[(data_month_ebill['Standardized_Date']>='2023-01-01')&(data_month_ebill['Standardized_Date']<'2024-01-01')]['Units_kWh'])/100, label='consumption', marker='o',color='b')
plt.plot(data_month_ebill[(data_month_ebill['Standardized_Date']>='2023-01-01')&(data_month_ebill['Standardized_Date']<'2024-01-01')]['Standardized_Date'], data_month_ebill[(data_month_ebill['Standardized_Date']>='2023-01-01')&(data_month_ebill['Standardized_Date']<'2024-01-01')]['CLEAR WATER PUMPING FLOW ML'], label='clear water', marker='o',color='y')
plt.legend()
plt.xticks(rotation=45)

In [ ]:
plt.plot(data_month_ebill[data_month_ebill['Standardized_Date']<'2023-01-01']['Standardized_Date'], data_month_ebill[data_month_ebill['Standardized_Date']<'2023-01-01']['CLEAR WATER PUMPING FLOW ML'], label='CLEAR WATER PUMPING FLOW ML', marker='o',color='g')
plt.ylabel('clear water pumping flow in ML')
plt.xticks(rotation=45)
plt.show()

In [ ]:
Specific energy consumption = kwh/treated water
unit cost = cost/treated water